In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
from datetime import date
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Merging training data from current and 2 months earlier
data_train = pd.read_csv("train_month_3_with_target.csv")
data_train_1 = pd.read_csv("train_month_1.csv")
data_train_2 = pd.read_csv("train_month_2.csv")
train_1 = pd.merge(data_train, data_train_1, on="client_id", suffixes=("", "_1"))
train=pd.merge(train_1, data_train_2, on="client_id", suffixes=("", "_2"))


In [3]:
# Merging test data from current and 2 months earlier
data_test = pd.read_csv("test_month_3.csv")
data_test_1 = pd.read_csv("test_month_1.csv")
data_test_2 = pd.read_csv("test_month_2.csv")
test_1 = pd.merge(data_test, data_test_1, on="client_id", suffixes=("", "_1"))
test=pd.merge(test_1, data_test_2, on="client_id", suffixes=("", "_2"))

In [4]:
# homebanking_active and has_homebanking are closely related 
# has 5 types of insurances 
# has 2 types of  personal loans 
# has 5 types of accounts (current, savings, pension, 2 starter ones)
# balances of 5 types of insurances 
# outstanding balances of 2 types of  personal loans 
# balances on 5 types of accounts (current, savings, pension, 2 starter ones)
# number of branches /and areas visited in the past month
# 2 types customer since (2 NAs)
# gender/ birthday / occupation (coded) (NAs)/ self employed
# education level (NAs)/ children (NAs) / relationship (NAs)


In [5]:
#for relationship lets take a third category (so these will be nominal)
#for children take the mode in the class
#for education take the mean
#for occupation code take the mode in the class
#for both customers take the mean in the class

In [6]:
# This will help to identify columns with the same values
###
#from itertools import combinations
#
#[(i, j) for i,j in combinations(x_train, 2) if x_train[i].equals(x_train[j])]
###

In [7]:
# Split the data in the training (70%) and validation set (30%)
x_train, x_valid, y_train, y_valid = train_test_split(train.drop(columns="target"), train['target'], test_size = .3, random_state = 10)

In [8]:
# This function drops the copy columns that have the same values, changes the date to time passed (months for 
# customer since, and years for age of the customer), then we also add new NA indicator columns, also 
# target is added in the end
def cleaning_tr (data_x, data_y):
    #DROPPING COPIES
    data_x.drop(columns=["customer_since_all_1", "customer_since_all_2", 
                        "customer_since_bank_1", "customer_since_bank_2", 
                        "customer_gender_1", "customer_gender_2",
                        "customer_birth_date_1", "customer_birth_date_2", 
                        "customer_postal_code_1", "customer_postal_code_2", 
                        "customer_occupation_code_1", "customer_occupation_code_2",
                        "customer_education_1", "customer_education_2", 
                       ], inplace=True)
    #DATE FORMATTING
    for column in ["customer_since_all", "customer_since_bank", "customer_birth_date"]:
        data_x[column]=data_x[column]+"-01"
        data_x[column]=pd.to_datetime(data_x[column], infer_datetime_format=True)
    t=data_x.index
    l=data_x.shape[0]
    for column in ["customer_since_all", "customer_since_bank", "customer_birth_date"]:
        a={'today':date.today()}
        x=pd.DataFrame(a, index=[0])
        y=pd.concat([x]*l, ignore_index=True)
        y.index=t
        z=pd.to_datetime(y["today"], infer_datetime_format=True)
        data_x[column]= (z-data_x[column])/np.timedelta64(1,'M')
    data_x["customer_birth_date"]=data_x["customer_birth_date"]/12
    #MAKING IS NA COLUMN
    for column in ["customer_since_all", "customer_since_bank", 
               "customer_occupation_code", "customer_education", 
               "customer_children", "customer_relationship", 
               "customer_children_1", "customer_relationship_1",
               "customer_children_2", "customer_relationship_2"]:
        data_x[column+"_is_na"]=data_x[column].isna().apply(lambda x: 0 if x==0 else 1)
        
    data_x["target"]=data_y
    
    return data_x


In [9]:
x_train_clean=cleaning_tr (x_train, y_train)


In [10]:
# This function makes new data out of clean data where we work on customer_relationship and customer_children
# we then drop target and client_id
def nahandle_tr(data_x):
    #FILLING IN NA COLUMNS
    import copy
    #we need deepcopy as we need the input of the function at a later point
    new=copy.deepcopy(data_x)

    #customer_relationship NA is given its own level
    new["customer_relationship"].fillna(value="unknown", inplace=True)
    new["customer_relationship_1"].fillna(value="unknown", inplace=True)
    new["customer_relationship_2"].fillna(value="unknown", inplace=True)
    new["customer_since_bank"].fillna(value=np.nanmean(new["customer_since_bank"]-new["customer_since_all"]), inplace=True)
    
    f = lambda x: x.mean() if np.issubdtype(x.dtype, np.number) else x.mode().iloc[0]
    new = new.fillna(new.groupby('target').transform(f))
    
    #FINAL TOUCH
    new=new.drop(columns=["target","client_id"])
    
    return new
    

In [11]:
x_train_handled=nahandle_tr(x_train_clean)

In [12]:
# now we only havw to encode customer relationships and drop the original columns
def encoding(data_x):
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(data_x[['customer_relationship']]).toarray())
    encoder_df.columns=["rel_a", "rel_b", "rel_c"]
    encoder_df.index=data_x.index
    data_x=data_x.join(encoder_df)
    encoder_df = pd.DataFrame(encoder.fit_transform(data_x[['customer_relationship_1']]).toarray())
    encoder_df.columns=["rel1_a", "rel1_b", "rel1_c"]
    encoder_df.index=data_x.index
    data_x=data_x.join(encoder_df)
    encoder_df = pd.DataFrame(encoder.fit_transform(data_x[['customer_relationship_2']]).toarray())
    encoder_df.columns=["rel2_a", "rel2_b", "rel2_c"]
    encoder_df.index=data_x.index
    data_x=data_x.join(encoder_df)
    
    #customer_childer are given numbers indicating how much care needed for kids
    region_dictionary = {'no': 0, 'onebaby' : 1, 'preschool':2, 'young':4, 'adolescent':5, 'grownup':6, 'mature':7, 'yes': 3}
    data_x['customer_children'] = data_x['customer_children'].apply(lambda x: region_dictionary[x])
    data_x['customer_children_1'] = data_x['customer_children_1'].apply(lambda x: region_dictionary[x])
    data_x['customer_children_2'] = data_x['customer_children_2'].apply(lambda x: region_dictionary[x])
    
    #FINAL TOUCH
    data_x=data_x.drop(columns=['customer_relationship', 'customer_relationship_1', 'customer_relationship_2'])
    
    return data_x


In [13]:
x_train_enc=encoding(x_train_handled)

In [14]:
val=pd.DataFrame(columns=["customer_children", "customer_children_1", "customer_children_2",
                          "customer_relationship", "customer_relationship_1", "customer_relationship_2",
                          "customer_since_all", "customer_since_bank", 
                          "customer_occupation_code", "customer_education"], index=[0,1])

for col in val.columns:
    val[col][0]=x_train_handled[col][x_train_handled[(x_train_clean[col].isna()) & (x_train_clean['target']==0)].index[0]]
    val[col][1]=x_train_handled[col][x_train_handled[(x_train_clean[col].isna()) & (x_train_clean['target']==1)].index[0]]



In [15]:
def normalise(data):
    from sklearn.preprocessing import StandardScaler
    non_binary=x_train_enc.loc[:,~x_train_enc.isin([0,1]).all()]
    scaler = StandardScaler()
    scaler.fit(non_binary)
    binary_1= data.loc[:,data.isin([0,1]).all()]
    non_binary_1=data.loc[:,~data.isin([0,1]).all()]
    trans=pd.DataFrame(scaler.transform(non_binary_1))
    trans.index=non_binary_1.index
    trans.columns=non_binary_1.columns
    return pd.concat([binary_1, trans], axis=1)

In [16]:
x_train=normalise(x_train_enc)

In [17]:
x_train.head()

,homebanking_active,has_homebanking,has_insurance_21,has_insurance_23,has_life_insurance_fixed_cap,has_life_insurance_decreasing_cap,has_fire_car_other_insurance,has_personal_loan,has_mortgage_loan,has_current_account,...,bal_personal_loan_2,bal_mortgage_loan_2,bal_current_account_2,bal_pension_saving_2,bal_savings_account_2,bal_savings_account_starter_2,bal_current_account_starter_2,visits_distinct_so_2,visits_distinct_so_areas_2,customer_children_2
19825,1,1,0,0,0,0,1,0,0,1,...,-0.151927,-0.251966,-0.408336,-0.109083,-1.105242,-0.065597,-0.073389,-0.458472,-0.190907,-0.560471
39136,0,0,0,0,0,0,0,0,0,0,...,-0.151927,-0.251966,-0.494115,-0.109083,-0.129402,-0.065597,-0.073389,-0.458472,-0.190907,0.295517
54454,0,0,0,0,0,0,0,0,0,0,...,-0.151927,-0.251966,-0.494115,-0.109083,-0.652234,-0.065597,-0.073389,-0.458472,-0.190907,-0.560471
55498,0,0,0,0,0,0,1,0,0,1,...,-0.151927,-0.251966,2.012126,-0.109083,-1.513119,-0.065597,-0.073389,-0.458472,-0.190907,1.579498
17011,0,0,0,0,0,0,0,0,0,0,...,-0.151927,-0.251966,-0.494115,-0.109083,-0.232435,-0.065597,-0.073389,-0.458472,-0.190907,-0.560471


## At this point we have x_train that has every column as numeric and without the target AND the values we need to impute, we will now proceed to cleaning/na_handling for the test/validation set


In [18]:
# This function drops the copy columns that have the same values, changes the date to time passed (months for 
# customer since, and years for age of the customer), then we also add new NA indicator columns
def cleaning_val (data):
    import copy
    data_x=copy.deepcopy(data)
    #DROPPING COPIES
    data_x.drop(columns=["customer_since_all_1", "customer_since_all_2", 
                        "customer_since_bank_1", "customer_since_bank_2", 
                        "customer_gender_1", "customer_gender_2",
                        "customer_birth_date_1", "customer_birth_date_2", 
                        "customer_postal_code_1", "customer_postal_code_2", 
                        "customer_occupation_code_1", "customer_occupation_code_2",
                        "customer_education_1", "customer_education_2", 
                       ], inplace=True)
    #DATE FORMATTING
    for column in ["customer_since_all", "customer_since_bank", "customer_birth_date"]:
        data_x[column]=data_x[column]+"-01"
        data_x[column]=pd.to_datetime(data_x[column], infer_datetime_format=True)
    t=data_x.index
    l=data_x.shape[0]
    for column in ["customer_since_all", "customer_since_bank", "customer_birth_date"]:
        a={'today':date.today()}
        x=pd.DataFrame(a, index=[0])
        y=pd.concat([x]*l, ignore_index=True)
        y.index=t
        z=pd.to_datetime(y["today"], infer_datetime_format=True)
        data_x[column]= (z-data_x[column])/np.timedelta64(1,'M')
    data_x["customer_birth_date"]=data_x["customer_birth_date"]/12
    #MAKING IS NA COLUMN
    for column in ["customer_since_all", "customer_since_bank", 
               "customer_occupation_code", "customer_education", 
               "customer_children", "customer_relationship", 
               "customer_children_1", "customer_relationship_1",
               "customer_children_2", "customer_relationship_2"]:
        data_x[column+"_is_na"]=data_x[column].isna().apply(lambda x: 0 if x==0 else 1)
    
    return data_x



In [19]:
# This function makes new data out of clean data where we work on customer_relationship and customer_children
# we then drop target and client_id
def nahandle_val(data):
    
    
    
    
    import copy
    data_x=copy.deepcopy(data)
    
    data_x=data_x.drop(columns=["client_id"])
    #FILLING IN NA COLUMNS
    
    for col in val.columns:
        #get dtype for column
        dt = data_x[col].dtype 
        #check if it is a number
        if dt == np.int64 or dt==np.float64:
            data_x[col].fillna(val[col].mean(), inplace=True)
        else:
            data_x[col].fillna(val[col].mode().iloc[0], inplace=True)
    

    
    return data_x

In [20]:
# now we only havw to encode customer relationships and drop the original columns
def encoding(data):
    import copy
    data_x=copy.deepcopy(data)
    
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder_df = pd.DataFrame(encoder.fit_transform(data_x[['customer_relationship']]).toarray())
    encoder_df.columns=["rel_a", "rel_b", "rel_c"]
    encoder_df.index=data_x.index
    data_x=data_x.join(encoder_df)
    encoder_df = pd.DataFrame(encoder.fit_transform(data_x[['customer_relationship_1']]).toarray())
    encoder_df.columns=["rel1_a", "rel1_b", "rel1_c"]
    encoder_df.index=data_x.index
    data_x=data_x.join(encoder_df)
    encoder_df = pd.DataFrame(encoder.fit_transform(data_x[['customer_relationship_2']]).toarray())
    encoder_df.columns=["rel2_a", "rel2_b", "rel2_c"]
    encoder_df.index=data_x.index
    data_x=data_x.join(encoder_df)
    
    #customer_childer are given numbers indicating how much care needed for kids
    region_dictionary = {'no': 0, 'onebaby' : 1, 'preschool':2, 'young':4, 'adolescent':5, 'grownup':6, 'mature':7, 'yes': 3}
    data_x['customer_children'] = data_x['customer_children'].apply(lambda x: region_dictionary[x])
    data_x['customer_children_1'] = data_x['customer_children_1'].apply(lambda x: region_dictionary[x])
    data_x['customer_children_2'] = data_x['customer_children_2'].apply(lambda x: region_dictionary[x])
    
    #FINAL TOUCH
    data_x=data_x.drop(columns=['customer_relationship', 'customer_relationship_1', 'customer_relationship_2'])
    
    return data_x



In [21]:
clean=cleaning_val(x_valid)

In [22]:
handled=nahandle_val(clean)

In [23]:
x_valid=normalise(encoding(handled))

In [24]:
clean=cleaning_val(test)

In [25]:
handled=nahandle_val(clean)

In [26]:
x_test=normalise(encoding(handled))

In [27]:
x_test.head()

,homebanking_active,has_homebanking,has_insurance_21,has_insurance_23,has_life_insurance_fixed_cap,has_life_insurance_decreasing_cap,has_fire_car_other_insurance,has_personal_loan,has_mortgage_loan,has_current_account,...,bal_personal_loan_2,bal_mortgage_loan_2,bal_current_account_2,bal_pension_saving_2,bal_savings_account_2,bal_savings_account_starter_2,bal_current_account_starter_2,visits_distinct_so_2,visits_distinct_so_areas_2,customer_children_2
0,0,0,0,0,0,0,0,0,0,1,...,-0.151927,-0.251966,-0.024195,-0.109083,-1.071181,-0.065597,-0.073389,-0.458472,-0.190907,-0.560471
1,0,0,0,0,0,0,1,0,0,1,...,-0.151927,-0.251966,-0.482927,-0.109083,-0.234990,-0.065597,-0.073389,-0.458472,-0.190907,-0.560471
2,0,0,0,0,0,1,1,0,1,1,...,-0.151927,4.093010,0.732899,-0.109083,-1.198057,-0.065597,-0.073389,-0.458472,-0.190907,2.435486
3,0,0,0,0,0,0,1,0,0,1,...,-0.151927,-0.251966,0.818678,-0.109083,0.828557,-0.065597,-0.073389,-0.458472,-0.190907,-0.560471
4,0,0,0,0,0,0,0,0,0,0,...,-0.151927,-0.251966,-0.494115,-0.109083,-0.645422,-0.065597,-0.073389,-0.458472,-0.190907,2.007492


## Now we have the data for val/test it is time to run some tree based models

In [28]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_features':[0.2, 0.4, 0.6, 0.8, 1], "sampling_strategy":["not majority", "not minority", "all"]}
# bagged decision trees with random undersampling for imbalanced classification
from imblearn.ensemble import BalancedBaggingClassifier
# define model
model = BalancedBaggingClassifier(n_estimators=10, verbose=2)
grids = GridSearchCV(model, parameters, verbose=2, cv=10)
grids.fit(x_train, y_train)

y_pred_prob = grids.predict_proba(x_valid)
print(roc_auc_score(y_valid, y_pred_prob[:,1]))

y_pred = grids.predict(x_valid)
print(confusion_matrix(y_valid, y_pred))


In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth':[5,10,20],"min_samples_split":[20, 30, 40, 50], "max_features":[20,30,50]}
# random forest for imbalanced classification
from sklearn.ensemble import RandomForestClassifier
# define model
model = RandomForestClassifier(n_estimators=50, class_weight='balanced', verbose=2)
grids = GridSearchCV(model, parameters, verbose=2, cv=10)
grids.fit(x_train, y_train)

y_pred_prob = grids.predict_proba(x_valid)
print(roc_auc_score(y_valid, y_pred_prob[:,1]))

y_pred = grids.predict(x_valid)
print(confusion_matrix(y_valid, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth':[10, 15, 20, 30, 35, 40, 45, 50], "min_samples_split":[5,10,20], "max_features":[10,20,30,50]}
# random forest with random undersampling for imbalanced classification
from imblearn.ensemble import BalancedRandomForestClassifier
# define model
model = BalancedRandomForestClassifier(n_estimators=50, class_weight='balanced', verbose=2)
grids = GridSearchCV(model, parameters, verbose=2)
grids.fit(x_train, y_train)
y_pred = grids.predict_proba(x_train)
print(roc_auc_score(y_train, y_pred[:,1]))
y_pred = grids.predict(x_valid)
print(confusion_matrix(y_valid, y_pred))


In [ ]:
y_pred = grids.predict_proba(x_valid)
print(roc_auc_score(y_valid, y_pred[:,1]))
y_pred = grids.predict(x_valid)
print(confusion_matrix(y_valid, y_pred))

In [30]:
from sklearn.linear_model import LogisticRegression
weights = np.linspace(0.9,0.99, 10)
parameters={'class_weight': [{0:x, 1:1.0-x} for x in weights],"C":[0.5, 0.6, 0.7, 0.8, 0.9, 1]}
model = LogisticRegression(verbose=2, penalty='l1', solver="saga", class_weight="balanced")
grids = GridSearchCV(model, parameters, verbose=3, cv=10)
grids.fit(x_train, y_train)

y_pred_prob = grids.predict_proba(x_valid)
print(roc_auc_score(y_valid, y_pred_prob[:,1]))

y_pred = grids.predict(x_valid)
print(confusion_matrix(y_valid, y_pred))

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 9 seconds
[CV 1/10] END ............................C=0.5;, score=0.698 total time=   9.7s


/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 9 seconds
[CV 2/10] END ............................C=0.5;, score=0.691 total time=   9.0s


...

/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 6 seconds
[CV 10/10] END .............................C=1;, score=0.680 total time=   7.0s


/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 8 seconds
0.7212416260936414
[[12899  5621]
 [  215   375]]


/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.4s finished


In [42]:
from sklearn.linear_model import LogisticRegression
parameters={"C":[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85,  0.9, 0.95, 1]}
model = LogisticRegression(verbose=2, penalty='l1', solver="saga", class_weight='balanced')
grids = GridSearchCV(model, parameters, verbose=3, cv=10)
grids.fit(x_train, y_train)

y_pred_prob = grids.predict_proba(x_valid)
print(roc_auc_score(y_valid, y_pred_prob[:,1]))

y_pred = grids.predict(x_valid)
print(confusion_matrix(y_valid, y_pred))

Fitting 10 folds for each of 11 candidates, totalling 110 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 9 seconds
[CV 1/10] END ............................C=0.5;, score=0.699 total time=   9.6s


/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 10 seconds
[CV 2/10] END ............................C=0.5;, score=0.691 total time=   9.7s


...

/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 9 seconds
[CV 10/10] END .............................C=1;, score=0.681 total time=   8.8s


/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


max_iter reached after 10 seconds
0.7212980927627485
[[12900  5620]
 [  216   374]]


/Users/muuratjann1/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.4s finished


In [43]:
y_pred = grids.predict_proba(x_test)

In [44]:
y_pred[:,1]

array([0.57351238, 0.38181938, 0.80634041, ..., 0.30186009, 0.37497792,
       0.4136508 ])

In [45]:
df=pd.DataFrame(y_pred[:,1])

In [46]:
df["ID"]=test["client_id"]

In [47]:
df.columns=["PROB", "ID"]

In [48]:
df = df[['ID', 'PROB']]

In [49]:
df

,ID,PROB
0,ccf4cd93d5c32cd8a59809d54b4d53ac,0.573512
1,56605a660d18549592653ff6941186f1,0.381819
2,bda5f84c05e5695a7ec10550b457890f,0.806340
3,a2f1c04bc3acf2222e658a897400798f,0.407005
4,e83aadc3b0d25dbc12a35551afa25807,0.351966
...,...,...
27295,1a634b5cfe6dbf35e77ce6519c0e3939,0.478669
27296,84b7af2ddbeb919ab238ef7e6a83936c,0.394291
27297,c257c09e3f4445d67b52a1007b0205df,0.301860
27298,fdd7a1b7e58f642a1fc4867b8f7cdf3a,0.374978


In [50]:
df.to_csv("resp_prediction", index=False)